In [ ]:
!git clone https://github.com/PhonePe/pulse.git

These are the required libraries to extract the data,Manipulation of data and cleaning and to store it in our sql

In [1]:
import os
import json
import pandas as pd
from datetime import datetime
import plotly.express as px
from sqlalchemy import create_engine
import mysql.connector
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)
mycursor.execute('use capstone_2')
username = 'root'
password = ''
host = 'localhost'  
database_name = 'capstone_2'
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}/{database_name}')


Extraction of the aggregated transaction data and data manipulation of that data using pandas data frame and storing that data in an sql database table

In [4]:
Data1={'State':[],'Year':[],'Quater':[],'Transaction':[],'Count':[],'Amount':[]}
path=r'C:\Users\kesav\Guvi\guvi main\capstone_2\pulse\data\aggregated\transaction\country\india\state'
t=os.listdir(path)
for i in t:
    p1=path+'/'+i+'/'
    t1=os.listdir(p1)
    for j in t1:
        p2=p1+'/'+j+'/'
        t2=os.listdir(p2)
        year = datetime.strptime(j, '%Y').year
        for k in t2:
            p3=p2+k
            d=open(p3)
            D=json.load(d)
            for t in D['data']['transactionData']:
                name=t['name']
                c=t['paymentInstruments'][0]['count']
                am=t['paymentInstruments'][0]['amount']
                Data1['State'].append(i)
                Data1['Year'].append(int(year))
                Data1['Quater'].append(int(k.strip('.json')))
                Data1['Transaction'].append(name)
                Data1['Count'].append(c)
                Data1['Amount'].append(am)
Df1=pd.DataFrame(Data1)
Df1['State'] = Df1['State'].str.replace('-&-', ' & ')
Df1['State'] = Df1['State'].str.replace('-Islands', '')
Df1['State'] = Df1['State'].str.replace('-', ' ')
def cap(s):
    return s.title()
Df1['State']=Df1['State'].apply(cap)
Df1['State'] = Df1['State'].str.replace('Islands', '')
#mycursor.execute('use capstone_2')
for i,row in Df1.iterrows():
    #here %S means string values 
    sql = "INSERT INTO t_1 VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    mydb.commit()

Extracting of data and creating the data and storing the data in the sql for the aggregated users information

In [7]:
Data_r={'State':[],'Year':[],'Quater':[],'Regestered_users':[],'AppOpens':[]}
State=[]
Year=[]
Quater=[]
Brand=[]
Count=[]
Percentage=[]
data2={'State':State,'Year':Year,'Quater':Quater,'Brand':Brand,'Count':Count,'Percentage':Percentage}
path1=r"C:\Users\kesav\Guvi\guvi main\capstone_2\pulse\data\aggregated\user\country\india\state"
t2_0=os.listdir(path1)#till state
for i in t2_0:
    p21=path1+'/'+i+'/'# select a state i
    t21=os.listdir(p21)
    for j in t21:
        p22=p21+'/'+j+'/'
        t22=os.listdir(p22)
        year2 = datetime.strptime(j, '%Y').year
        for k in t22:
            p23=p22+'/'+k # select quater json file
            d2=open(p23)
            D2=json.load(d2)
            for r in D2:
                r_u=D2['data']['aggregated']["registeredUsers"]
                r_a=D2['data']['aggregated']['appOpens']
                Data_r['Regestered_users'].append(r_u)
                Data_r['AppOpens'].append(r_a)
                Data_r['State'].append(i)
                Data_r['Year'].append(int(year2))
                Data_r['Quater'].append(int(k.strip('.json')))
        for k in t22:
            p23=p22+'/'+k # select quater json file
            d3=open(p23)
            D3=json.load(d3)
            try:
                for u in D3['data']['usersByDevice']:
                            Brand.append(u['brand'])
                            Count.append(u['count'])
                            Percentage.append(u['percentage'])
                            State.append(i)
                            Year.append(j)
                            Quater.append(int(k.strip('.json')))
            except:
                pass 
Df2=pd.DataFrame(Data_r)
Df2.drop_duplicates(inplace=True, keep='last')
Df2.reset_index(drop=True, inplace=True)
Df2['State'] = Df2['State'].str.replace('-&-', ' & ')
Df2['State'] = Df2['State'].str.replace('-', ' ')
#Df_2['Year']=pd.to_datetime(Df_2['Year'],format='%Y')
def cap(s):
    return s.title()
Df2['State']=Df2['State'].apply(cap)
Df2['State'] = Df2['State'].str.replace('Islands','')
for i,row in Df2.iterrows():
    #here %S means string values 
    sql = "INSERT INTO t_2 VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    # the connection is not auto committed by default, so we must commit to save our changes
    mydb.commit()
#table_name = 't_2'
#Df2.to_sql(table_name, con=engine, if_exists='append', index=False)
for i in range (len(State)):
    mycursor.execute('use capstone_2')
    mycursor.execute('insert into t_3 (State,Year,Quater,Brand,Count,Percentage)values(%s,%s,%s,%s,%s,%s)',(State[i],Year[i],Quater[i],Brand[i],Count[i],Percentage[i]))
    mydb.commit()


map data transaction

Creating code for extracting the data and storing it in Panda's data frame and clearing the data in a required format and storing it into an sql database for the map visualisation

In [2]:
Datag1={'State':[],'Year':[],'Quater':[],'District':[],'Count':[],'Amount':[]}
path4=r"C:\Users\kesav\Guvi\guvi main\capstone_2\pulse\data\map\transaction\hover\country\india\state"
tg=os.listdir(path4)
for  i in tg:
    pg1=path4+'/'+i+'/'
    tg1=os.listdir(pg1)
    for j in tg1:
        pg2=pg1+'/'+j+'/'
        tg2=os.listdir(pg2)
        year_g1 = datetime.strptime(j, '%Y').year
        for k in tg2:
            pg3=pg2+k
            dg=open(pg3)
            Dg1=json.load(dg)
            for t in Dg1['data']['hoverDataList']:
                name_d=t['name']
                cg=t['metric'][0]['count']
                ag=t['metric'][0]['amount']
                Datag1['State'].append(i)
                Datag1['Year'].append(int(year_g1))
                Datag1['Quater'].append(int(k.strip('.json')))
                Datag1['District'].append(name_d)
                Datag1['Count'].append(cg)
                Datag1['Amount'].append(ag)
           
Df1_g=pd.DataFrame(Datag1)
def cap(s):
    return s.title()
Df1_g['State']=Df1_g['State'].apply(cap)
Df1_g['District'] = Df1_g['District'].str.replace('district', '')
Df1_g['State'] = Df1_g['State'].str.replace('-&-', ' & ')
Df1_g['State'] = Df1_g['State'].str.replace('Islands', '')
Df1_g['State'] = Df1_g['State'].str.replace('-', ' ')

# insert into tg_1 table
for i,row in Df1_g.iterrows():
#here %S means string values 
    sql = "INSERT INTO tg_1 VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()



map users

Extracting the data and cleaning the data and storing the data in a sql table for map visualisation of users data

In [3]:
Datagu1={'State':[],'Year':[],'Quater':[],'District':[],"registeredUsers":[],"AppOpens":[]}
pathm_u=r'C:\Users\kesav\Guvi\guvi main\capstone_2\pulse\data\map\user\hover\country\india\state'
tmu=os.listdir(pathm_u)
for i in tmu:
    pmu1=pathm_u+'/'+i+'/'
    tmu1=os.listdir(pmu1)
    for j in tmu1:
        pmu2=pmu1+'/'+j+'/'
        tmu2=os.listdir(pmu2)
        year_mu1 = datetime.strptime(j, '%Y').year
        for k in tmu2:
            pmu3=pmu2+k
            dmu=open(pmu3)
            Dmu1=json.load(dmu)
            for district, data in Dmu1["data"]["hoverData"].items():
                Datagu1['State'].append(i)
                Datagu1['Year'].append(int(year_mu1))
                Datagu1['Quater'].append(int(k.strip('.json')))
                Datagu1['District'].append(district)
                Datagu1['registeredUsers'].append(data["registeredUsers"])
                Datagu1['AppOpens'].append(data["appOpens"])

Dfm_u=pd.DataFrame(Datagu1)
def cap(s):
    return s.title()
Dfm_u['State']=Dfm_u['State'].apply(cap)
Dfm_u['District'] = Dfm_u['District'].str.replace('district', '')
Dfm_u['State'] = Dfm_u['State'].str.replace('-&-', ' & ')
Dfm_u['State'] = Dfm_u['State'].str.replace('-', ' ')
Dfm_u['State'] = Dfm_u['State'].str.replace('Islands', '')
State=Dfm_u['State']
Year=Datagu1['Year']
Quater=Datagu1['Quater']
District=Dfm_u['District']
registered_users=Datagu1['registeredUsers']
AppOpens=Datagu1['AppOpens']
# insert into tg_2 table
for i,row in Dfm_u.iterrows():
    #here %S means string values 
    sql = "INSERT INTO tg_2 VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()


top

Extracting the data and cleaning the data and storing it in a sql table for top transactions of districts data

In [74]:
import mysql.connector
mydb = mysql.connector.connect(host="localhost",user="root",password="")
mycursor = mydb.cursor(buffered=True)
mycursor.execute('use capstone_2')

In [85]:
Data5={'State':[],'Year':[],'Quater':[],'Pincode':[],'Count':[],'Amount':[]}
data5={'State':[],'Year':[],'Quater':[],'District':[],'count':[],'amount':[]}
path5=r'C:\Users\kesav\Guvi\guvi main\capstone_2\pulse\data\top\transaction\country\india\state'
t5=os.listdir(path5)
for i in t5:
    p1_5=path5+'/'+i+'/'
    t1_5=os.listdir(p1_5)
    for j in t1_5:
        p2_5=p1_5+'/'+j+'/'
        t2_5=os.listdir(p2_5)
        year5 = datetime.strptime(j, '%Y').year
        for k in t2_5:
            p3_5=p2_5+k
            d_5=open(p3_5)
            D_5=json.load(d_5)
            for t in D_5['data']['districts']:
                
                data5['State'].append(i)
                data5['Year'].append(year5)
                data5['Quater'].append(int(k.strip('.json')))
                data5['District'].append(t['entityName'])
                data5['count'].append(t['metric']['count'])
                data5['amount'].append(t['metric']['amount'])
            for t in D_5['data']['pincodes']:
                Data5['State'].append(i)
                Data5['Year'].append(year5)
                Data5['Quater'].append(int(k.strip('.json')))
                Data5['Pincode'].append(t['entityName'])
                Data5['Count'].append(t['metric']['count'])
                Data5['Amount'].append(t['metric']['amount'])
            
Df_5=pd.DataFrame(data5)
Df_5['State'] = Df_5['State'].str.replace('-&-', ' & ')
Df_5['State'] = Df_5['State'].str.replace('Islands', '')
Df_5['State'] = Df_5['State'].str.replace('-', ' ')
def cap(s):
    return s.title()
Df_5['State']=Df_5['State'].apply(cap)
Df_5['State'] = Df_5['State'].str.replace('Islands', '')
for i,row in Df_5.iterrows():
    #here %S means string values 
    sql = "INSERT INTO t_4 VALUES (%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()
Df_6=pd.DataFrame(Data5)
Df_6=pd.DataFrame(Data5)
Df_6['State'] = Df_6['State'].str.replace('-&-', ' & ')
Df_6['State'] = Df_6['State'].str.replace('Islands', '')
Df_6['State'] = Df_6['State'].str.replace('-', ' ')
def cap(s):
    return s.title()
Df_6['State']=Df_6['State'].apply(cap)
Df_6['State'] = Df_6['State'].str.replace('Islands', '')
table_name = 't_5'
Df_6.to_sql(table_name, con=engine, if_exists='append', index=False)



8567

Extracting and transforming the data and storing it in the sql table of top registered users

In [2]:
data6={'State':[],'Year':[],'Quater':[],'District':[],'registeredUsers':[]}
path6=r'C:\Users\kesav\Guvi\guvi main\capstone_2\pulse\data\top\user\country\india\state'
t6=os.listdir(path6)
for i in t6:
    p1_6=path6+'/'+i+'/'
    t1_6=os.listdir(p1_6)
    for j in t1_6:
        p2_6=p1_6+'/'+j+'/'
        t2_6=os.listdir(p2_6)
        year6 = datetime.strptime(j, '%Y').year
        for k in t2_6:
            p3_6=p2_6+k
            d_6=open(p3_6)
            D_6=json.load(d_6)
            for t in D_6['data']['districts']: 
                data6['State'].append(i)
                data6['Year'].append(year6)
                data6['Quater'].append(int(k.strip('.json')))
                data6['District'].append(t["name"])
                data6['registeredUsers'].append(t["registeredUsers"])
Df_7=pd.DataFrame(data6)
def cap(s):
    return s.title()
Df_7['State']=Df_7['State'].apply(cap)               
Df_7['State'] = Df_7['State'].str.replace('-&-', ' & ')
Df_7['State'] = Df_7['State'].str.replace('Islands', '')
Df_7['State'] = Df_7['State'].str.replace('-', ' ')
Df_7['State'] = Df_7['State'].str.replace('Islands', '')
table_name = 't_6'
Df_7.to_sql(table_name, con=engine, if_exists='append', index=False)    

7104